# **Migrating Semantic Models and Reports between Workspaces**

This notebook uses the native `semantic-link-labs` functions to:
- Migrate semantic models between workspaces
- Clone reports between workspaces

**Documentation:**
- [deploy_semantic_model](https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.html#sempy_labs.deploy_semantic_model)
- [clone_report](https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.report.html)

## 1. Install Semantic Link Labs

In [ ]:
%pip install semantic-link-labs --upgrade

## 2. Import libraries

In [ ]:
import sempy_labs as labs
from sempy_labs.report import clone_report
import sempy.fabric as fabric

## 3. Workspace and Resource Configuration

Define the IDs/names of your source and target workspaces:

In [ ]:
# Source workspace
SOURCE_WORKSPACE = "XXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX"  # Can be name or ID

# Target workspace
TARGET_WORKSPACE = "XXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX"  # Can be name or ID

# Semantic Model (Dataset)
SOURCE_DATASET_NAME = "Gold_Semantic"  # Name of the source dataset
TARGET_DATASET_NAME = "Gold_Semantic_Migrated"  # Name of the dataset in target

# Report (optional)
SOURCE_REPORT_NAME = "Report"  # Name of the report to clone
TARGET_REPORT_NAME = "Reporte_Cloned"  # Name of the cloned report

## 4. Migrate Semantic Model

Use `deploy_semantic_model` to copy a semantic model from one workspace to another.

**Main parameters:**
- `source_dataset`: Name of the source semantic model
- `source_workspace`: Source workspace (name or ID)
- `target_dataset`: Name of the new semantic model in target
- `target_workspace`: Target workspace (name or ID)
- `refresh_target_dataset`: If True, refreshes the dataset after creating it (default: True)
- `overwrite`: If True, overwrites the dataset if it already exists (default: False)

In [ ]:
# Option 1: Migrate semantic model WITHOUT overwriting if it already exists
try:
    labs.deploy_semantic_model(
        source_dataset=SOURCE_DATASET_NAME,
        source_workspace=SOURCE_WORKSPACE,
        target_dataset=TARGET_DATASET_NAME,
        target_workspace=TARGET_WORKSPACE,
        refresh_target_dataset=True,  # Refresh after creating
        overwrite=False  # Don't overwrite if already exists
    )
    print(f"✅ Semantic model '{TARGET_DATASET_NAME}' migrated successfully")
except Exception as e:
    print(f"❌ Error migrating semantic model: {str(e)}")

In [ ]:
# Option 2: Migrate semantic model OVERWRITING if it already exists
try:
    labs.deploy_semantic_model(
        source_dataset=SOURCE_DATASET_NAME,
        source_workspace=SOURCE_WORKSPACE,
        target_dataset=TARGET_DATASET_NAME,
        target_workspace=TARGET_WORKSPACE,
        refresh_target_dataset=True,
        overwrite=True  # OVERWRITE if already exists
    )
    print(f"✅ Semantic model '{TARGET_DATASET_NAME}' migrated/updated successfully")
except Exception as e:
    print(f"❌ Error migrating semantic model: {str(e)}")

## 5. Verify Migrated Semantic Model

List the datasets in the target workspace to confirm the migration:

In [ ]:
# List datasets in the target workspace
datasets = fabric.list_datasets(workspace=TARGET_WORKSPACE)
print(f"\nDatasets in target workspace:")
print(datasets[['Dataset Name', 'Dataset ID', 'Created Timestamp']])

## 6. Clone Report (Optional)

Use `clone_report` to clone a report from one workspace to another.

**Main parameters:**
- `report`: Name of the source report
- `cloned_report`: Name of the cloned report
- `workspace`: Workspace where the original report is located
- `target_workspace`: Target workspace for the cloned report
- `target_dataset`: Name of the semantic model that the cloned report will use
- `target_dataset_workspace`: Workspace where the target semantic model is located

In [ ]:
# Clone report to the target workspace connecting it to the migrated semantic model
try:
    clone_report(
        report=SOURCE_REPORT_NAME,
        cloned_report=TARGET_REPORT_NAME,
        workspace=SOURCE_WORKSPACE,  # Workspace where the original report is located
        target_workspace=TARGET_WORKSPACE,  # Target workspace for the report
        target_dataset=TARGET_DATASET_NAME,  # Connect to the migrated semantic model
        target_dataset_workspace=TARGET_WORKSPACE  # Workspace of the semantic model
    )
    print(f"✅ Report '{TARGET_REPORT_NAME}' cloned successfully")
except Exception as e:
    print(f"❌ Error cloning report: {str(e)}")

## 7. Verify Cloned Report

In [ ]:
# List reports in the target workspace
reports = fabric.list_reports(workspace=TARGET_WORKSPACE)
print(f"\nReports in target workspace:")
print(reports[['Name', 'Id']])

## 8. Helper Function for Complete Migration

A function that combines semantic model migration and report cloning:

In [ ]:
def migrate_complete_solution(
    source_workspace: str,
    target_workspace: str,
    source_dataset: str,
    target_dataset: str,
    source_report: str = None,
    target_report: str = None,
    overwrite_dataset: bool = False,
    refresh_dataset: bool = True
):
    """
    Migrates a semantic model and optionally a report from one workspace to another.
    
    Args:
        source_workspace: Source workspace (name or ID)
        target_workspace: Target workspace (name or ID)
        source_dataset: Name of the source semantic model
        target_dataset: Name of the target semantic model
        source_report: Name of the report to clone (optional)
        target_report: Name of the cloned report (optional)
        overwrite_dataset: If True, overwrites the dataset if it exists
        refresh_dataset: If True, refreshes the dataset after creating it
    
    Returns:
        dict: Operation result
    """
    results = {
        'dataset_migrated': False,
        'report_cloned': False,
        'errors': []
    }
    
    # 1. Migrate semantic model
    print(f"🔄 Migrating semantic model '{source_dataset}'...")
    try:
        labs.deploy_semantic_model(
            source_dataset=source_dataset,
            source_workspace=source_workspace,
            target_dataset=target_dataset,
            target_workspace=target_workspace,
            refresh_target_dataset=refresh_dataset,
            overwrite=overwrite_dataset
        )
        results['dataset_migrated'] = True
        print(f"✅ Semantic model migrated: '{target_dataset}'")
    except Exception as e:
        error_msg = f"Error migrating semantic model: {str(e)}"
        results['errors'].append(error_msg)
        print(f"❌ {error_msg}")
        return results
    
    # 2. Clone report (if specified)
    if source_report and target_report:
        print(f"\n🔄 Cloning report '{source_report}'...")
        try:
            clone_report(
                report=source_report,
                cloned_report=target_report,
                workspace=source_workspace,
                target_workspace=target_workspace,
                target_dataset=target_dataset,
                target_dataset_workspace=target_workspace
            )
            results['report_cloned'] = True
            print(f"✅ Report cloned: '{target_report}'")
        except Exception as e:
            error_msg = f"Error cloning report: {str(e)}"
            results['errors'].append(error_msg)
            print(f"❌ {error_msg}")
    
    return results

## 9. Use the Helper Function

In [ ]:
# Example: Execute the helper function to migrate both semantic model AND report
# This will run the complete migration process
result = migrate_complete_solution(
    source_workspace=SOURCE_WORKSPACE,
    target_workspace=TARGET_WORKSPACE,
    source_dataset=SOURCE_DATASET_NAME,
    target_dataset=TARGET_DATASET_NAME,
    source_report=SOURCE_REPORT_NAME,
    target_report=TARGET_REPORT_NAME,
    overwrite_dataset=False,
    refresh_dataset=True
)

# Display the migration summary
print("\n" + "="*50)
print("MIGRATION SUMMARY")
print("="*50)
print(f"Dataset migrated: {result['dataset_migrated']}")
print(f"Report cloned: {result['report_cloned']}")
if result['errors']:
    print(f"\nErrors found:")
    for error in result['errors']:
        print(f"  - {error}")

## 10. List Reports Using a Semantic Model

Useful to identify which reports depend on a semantic model:

In [ ]:
# List reports that use the source semantic model
reports_using_dataset = labs.list_reports_using_semantic_model(
    dataset=SOURCE_DATASET_NAME,
    workspace=SOURCE_WORKSPACE
)

print(f"Reports that use the semantic model '{SOURCE_DATASET_NAME}':")
print(reports_using_dataset)